In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

## Data Acquisition
www.baseball-reference.com only allows 20 requests per minute. If we violate this rule, our session will be locked out for one hour.

In [2]:
# request website for each year
main_url = 'https://www.baseball-reference.com'
years = ['2019', '2021', '2022']
pages = []
soups = []
for i in range(len(years)):
    url = f'{main_url}/leagues/majors/{years[i]}.shtml'
    page = requests.get(url)
    pages.append(page)
    soups.append(BeautifulSoup(page.content))
    
# three requests

In [3]:
# loop through soups to get the winning team links
pattern = '/teams/[A-Z][A-Z][A-Z]/20[0-9][0-9].shtml'
winning_team_links = []
for k in range(len(soups)):
    hrefs = soups[k].find_all('a', href=True)
    for i in range(len(hrefs)):
        new_str = str(hrefs[i])
        check = re.search(pattern, new_str)
        if check:
            link = new_str[9:30]  
            break
    winning_team_links.append(link)

In [4]:
# request website for each winning team
win_team_pages = []
win_team_soups = []
for i in range(len(winning_team_links)):
    url = f'{main_url}{winning_team_links[i]}'
    page = requests.get(url)
    win_team_pages.append(page)
    win_team_soups.append(BeautifulSoup(page.content))
    
# three requests

In [7]:
# create pandas dataframes for team stats that won the world series
batting_team_stats = []
pitching_team_stats = []
for i in range(len(win_team_pages)):
    [batting, pitching] = pd.read_html(win_team_pages[i].content)
    batting_team_stats.append(batting)
    pitching_team_stats.append(pitching)

# need to delete row index 8 in most (if not all) dataframes for batting. Also delete the last 5 rows.
# delete index row 5, 11, and last 3 rows in pitching. Look at each dataframe before making edits.

In [41]:
# loop through table to get link for each player of each team that won World Series
player_links = []
for k in range(len(win_team_soups)):
    team_pl_links = []
    counter =0
    for this_soup in win_team_soups[k].find_all('td', {"class": "left", "data-stat": "player"}):
        try:
            counter += 1
            link = this_soup.find('a')['href']
            team_pl_links.append(link)
            if counter >= 9:
                break
        except:
            pass
    player_links.append(team_pl_links)

In [42]:
print(player_links[0])

['/players/g/gomesya01.shtml', '/players/a/adamsma01.shtml', '/players/d/doziebr01.shtml', '/players/t/turnetr01.shtml', '/players/r/rendoan01.shtml', '/players/s/sotoju01.shtml', '/players/r/roblevi01.shtml', '/players/e/eatonad02.shtml', '/players/k/kendrho01.shtml']
